In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt

In [2]:
df = pd.read_excel('C:/Users/user/OneDrive/Desktop/Assignments/Blackcoffer/Input.xlsx')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   URL_ID  114 non-null    float64
 1   URL     114 non-null    object 
dtypes: float64(1), object(1)
memory usage: 1.9+ KB


,URL_ID,URL
0,123.0,https://insights.blackcoffer.com/rise-of-telem...
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...
4,432.0,https://insights.blackcoffer.com/rise-of-telem...


In [3]:
import newspaper
from newspaper import Article
 
# Assign single url
url ='https://insights.blackcoffer.com/impact-of-covid-19-pandemic-on-office-space-and-co-working-industries/'
 
# Extract web data
url_i = newspaper.Article(url="%s" % (url), language='en')
url_i.download()
url_i.parse()
print(url_i.text)

COVID 19 has bought the world to its knees. With businesses being shut, travel being banned, schools, and colleges being closed, we have observed an impeccable amount of sorrow and despise along with a great amount of mental torture and destruction. Though situations are improving now, and the lockdowns are being lifted, it is a fact that it cannot be shunned that this virus will have a long term impact on people, and the effects will not only be felt economically and physically but also mentally.

We’ve been reading articles regarding the impact of COVID 19 on the economy, education, mental health, and every other possible aspect. But one topic which has not been talked about much is the impact of COVID 19 on office space and co-working industries. Well, it is quite obvious that offices will start functioning now, but, the COVID period hasn’t ended. This means that social distancing will also have to be followed in offices; where it will be a problem to do so because of the lack of ad

In [4]:
articles=[]
for url in df['URL']:
    try:
        url_i = newspaper.Article(url="%s" % (url), language='en')
        url_i.download()
        url_i.parse()
        articles.append(url_i.text)
    except:
        articles.append('')   
df['text']=articles
df.head(10)

,URL_ID,URL,text
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,"Telemedicine, the use of technology to diagnos..."
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,"The rise of e-health, or the use of electronic..."
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,2020 was the year the world was ravaged by the...
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,"“More gains on quality, affordability and acce..."
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,"“More gains on quality, affordability and acce..."
5,2893.8,https://insights.blackcoffer.com/rise-of-chatb...,The human race is known to come up with invent...
6,3355.6,https://insights.blackcoffer.com/rise-of-e-hea...,Introduction\n\nThe phrase “e-health” refers t...
7,3817.4,https://insights.blackcoffer.com/how-does-mark...,Marketing can be defined as an act of promotio...
8,4279.2,https://insights.blackcoffer.com/how-advertise...,Advertising allows companies to differentiate ...
9,4741.0,https://insights.blackcoffer.com/negative-effe...,"“Society can exist without Marketing, but Mark..."


### Text Cleaning

In [6]:
import re
import string
import os
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [9]:
# Tokenizer

def transform(text):
    text = re.sub(r'[^a-zA-Z-]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    texts =text.split()
    filtered_tokens = [word for word in texts if word.lower() not in stopwords.words('english')]
    return' '.join(filtered_tokens) 

df['t_text']=df['text'].apply(transform)
df.t_text

0      Telemedicine use technology diagnose treat pat...
1      rise e-health use electronic means facilitate ...
2      year world ravaged SarsCov virus notorious vir...
3      gains quality affordability accessibility way ...
4      gains quality affordability accessibility way ...
                             ...                        
109    jumping topic would like give overview Coronav...
110    coronavirus spreads around world countries imp...
111    Alibaba Ping Google Ford companies around glob...
112    British ruled India many Indians accepted work...
113    business business longer business increase bot...
Name: t_text, Length: 114, dtype: object

## Average Number of Words Per Sentence

In [54]:
# Calculate Word Count
df['word_count'] = df['t_text'].apply(lambda x: len(str(x).split()))

def Average_words_per_sentence(df):
    word_count = df['word_count']       
    sentence_count = len(nltk.sent_tokenize(df['text']))
    if sentence_count != 0:
        return word_count / sentence_count
    else:
        return 0

df['Average_number_of_words_per_sentence'] = df.apply(Average_words_per_sentence, axis=1)
df.Average_number_of_words_per_sentence

0      12.512500
1      13.280000
2       8.884058
3      12.433333
4      12.433333
         ...    
109    12.837838
110    21.420000
111    14.585714
112    13.237288
113    14.553191
Name: Average_number_of_words_per_sentence, Length: 114, dtype: float64

## Average Words Length 

In [55]:
Character_length = df['t_text'].apply(lambda x: sum(len(word) for word in x))
df['Avg_Word_length'] = np.nan

for i in range(0,len(df)):
    df['Avg_Word_length'] = df['t_text'].apply(lambda x: sum(len(word) for word in x))/df['t_text'].apply(lambda x: len(str(x).split()))
    
df.Avg_Word_length

0      8.387612
1      8.704819
2      8.109299
3      8.183646
4      8.183646
         ...   
109    7.802105
110    7.555556
111    7.946131
112    7.718310
113    8.559942
Name: Avg_Word_length, Length: 114, dtype: float64

## Complex Word Count

In [12]:
def complex_word_count(text):
    vowels = 'aeiou'                  # Required syllables
    count = 0
    for i in text.split():
        syllables = 0
        for j in i:
            if j in vowels:
                syllables += 1
        if syllables > 2:
            count += 1
    return count

df['complex_word_count'] = df['t_text'].apply(complex_word_count)
df.complex_word_count

0      588
1      211
2      317
3      395
4      395
      ... 
109    214
110    412
111    490
112    347
113    384
Name: complex_word_count, Length: 114, dtype: int64

## Syllable Count per Word

In [56]:
def Syllable_Count_Per_Word(text):
    vowels = 'aeiou'
    word_count = len(text.split())
    total_syllables = 0

    for word in text.split():
        if re.search(r'\b\w+es\b|\b\w+ed\b', word):
            continue
        else:
            syllables = 0
            for i in word:
                if i in vowels:
                    syllables += 1
            total_syllables += syllables

    if word_count > 0:
        return total_syllables / word_count
    else:
        return 0

df['Syllable_Count_Per_Word'] = df['t_text'].apply(Syllable_Count_Per_Word)
df.Syllable_Count_Per_Word

0      2.678322
1      2.629518
2      2.199021
3      2.325737
4      2.325737
         ...   
109    2.176842
110    2.100840
111    2.245837
112    2.183099
113    2.396199
Name: Syllable_Count_Per_Word, Length: 114, dtype: float64

## Personal Pronoun Count

In [14]:
df['personal_pronoun_count']=np.nan

for i in range(0,len(df)):
    df['personal_pronoun_count'] = df['text'].apply(lambda x: sum(1 for word in nltk.word_tokenize(x) if word.lower() in ['i', 'we', 'my', 'ours', 'us']))

df.personal_pronoun_count

0      2
1      3
2      3
3      7
4      7
      ..
109    1
110    3
111    0
112    8
113    1
Name: personal_pronoun_count, Length: 114, dtype: int64

## Analysis of Readability 

In [57]:
def calculation_metrics(df):
    word_count = df['word_count']
    sentence_count = len(nltk.sent_tokenize(df['text']))
    complex_word_count = df['complex_word_count']

    if sentence_count != 0:
        df['Average_Sentence_Length'] = word_count / sentence_count
    else:
        df['Average_Sentence_Length'] = 0

    if word_count != 0:
        df['Percentage_of_Complex_words'] = complex_word_count / word_count
    else:
        df['Percentage_of_Complex_words'] = 0

    df['Fog_Index'] = 0.4 * (df['Average_Sentence_Length'] + df['Percentage_of_Complex_words'])

    return df

df = df.apply(calculation_metrics, axis=1)


In [58]:
df.Average_Sentence_Length

0      12.512500
1      13.280000
2       8.884058
3      12.433333
4      12.433333
         ...    
109    12.837838
110    21.420000
111    14.585714
112    13.237288
113    14.553191
Name: Average_Sentence_Length, Length: 114, dtype: float64

In [59]:
df.Percentage_of_Complex_words

0      0.587413
1      0.635542
2      0.517129
3      0.529491
4      0.529491
         ...   
109    0.450526
110    0.384687
111    0.479922
112    0.444302
113    0.561404
Name: Percentage_of_Complex_words, Length: 114, dtype: float64

In [60]:
df.Fog_Index

0      5.239965
1      5.566217
2      3.760475
3      5.185130
4      5.185130
         ...   
109    5.315346
110    8.721875
111    6.026254
112    5.472636
113    6.045838
Name: Fog_Index, Length: 114, dtype: float64

## Sentimental Analysis

In [19]:
file_path="C:/Users/user/OneDrive/Desktop/Assignments/Blackcoffer/StopWords"
def read_text_file(file_path):
    with open(file_path, 'r') as f:
        print(f.read()) 

In [20]:
stop_words = []
for i in os.listdir(file_path):
    with open(os.path.join(file_path, i), 'r') as f:
        words = f.read().splitlines()
        stop_words.extend(words)

In [24]:
stop_words = [word.lower() for word in stop_words] 
stopword = nltk.corpus.stopwords.words('english')
stop_words.extend(stopword)     

In [25]:
# Text cleaning using updated stopwords
def transform(text):
    text = re.sub(r'[^a-zA-Z-]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    tokens = word_tokenize(text.lower())
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return filtered_tokens

df['s_text']=df['text'].apply(transform)
df.s_text

0      [telemedicine, technology, diagnose, patients,...
1      [rise, e-health, electronic, facilitate, healt...
2      [world, ravaged, sarscov, virus, notorious, vi...
3      [gains, quality, affordability, accessibility,...
4      [gains, quality, affordability, accessibility,...
                             ...                        
109    [jumping, topic, give, overview, coronavirus, ...
110    [coronavirus, spreads, world, countries, imple...
111    [alibaba, google, companies, globe, telling, s...
112    [ruled, indians, accepted, work, policy, rebel...
113    [business, business, longer, business, increas...
Name: s_text, Length: 114, dtype: object

In [26]:
sentiment="C:/Users/user/OneDrive/Desktop/Assignments/Blackcoffer/MasterDictionary"
def read_text_file(sentiment):
    with open(sentiment, 'r') as word:
        print(word.read())

In [27]:
positive=set()
negative=set()

for i in os.listdir(sentiment):
    if i =='positive-words.txt':
        with open(os.path.join(sentiment,i),'r') as f:
          positive.update(set(f.read().splitlines()))
    else:
        with open(os.path.join(sentiment,i),'r') as f:
          negative.update(set(f.read().splitlines()))

### Positive/Negative Words

In [28]:
Positive_words=[]
Negative_words=[]
for i in range(0,len(df['s_text'])):
  Positive_words.append([word for word in df['s_text'][i] if word.lower() in positive])
  Negative_words.append([word for word in df['s_text'][i] if word.lower() in negative])        

### Positive/Negative Score

In [29]:
Positive_score=[]
Negative_score=[]
for i in range(0,len(df['s_text'])):
    Positive_score.append(len(Positive_words[i]))
    Negative_score.append(len(Negative_words[i]))

### Polarity/ Subjectivity Score

In [30]:
# Polarity Score = (Positive Score – Negative Score)/ ((Positive Score + Negative Score) + 0.000001)
# Subjectivity Score = (Positive Score + Negative Score)/ ((Total Words after cleaning) + 0.000001)

Polarity_score=[]
Subjectivity_score=[]
for i in range(0,len(df['t_text'])):
    Polarity_score.append((Positive_score[i] - Negative_score[i]) / ((Positive_score[i] + Negative_score[i]) + 0.000001))
    Subjectivity_score.append((Positive_score[i] + Negative_score[i]) / ((len(df['t_text'][i])) + 0.000001))
    

In [31]:
df['Positive_score']= Positive_score
df['Negative_score']= Negative_score
df['Polarity_score']= Polarity_score
df['Subjectivity_score']= Subjectivity_score

In [32]:
df.Positive_score

0      80
1      38
2      18
3      35
4      35
       ..
109     5
110    21
111    85
112    26
113    50
Name: Positive_score, Length: 114, dtype: int64

In [33]:
df.Negative_score

0      23
1      13
2      27
3      26
4      26
       ..
109    40
110    62
111    32
112    21
113    36
Name: Negative_score, Length: 114, dtype: int64

In [34]:
df.Polarity_score

0      0.553398
1      0.490196
2     -0.200000
3      0.147541
4      0.147541
         ...   
109   -0.777778
110   -0.493976
111    0.452991
112    0.106383
113    0.162791
Name: Polarity_score, Length: 114, dtype: float64

In [35]:
df.Subjectivity_score

0      0.012268
1      0.017647
2      0.009053
3      0.009992
4      0.009992
         ...   
109    0.012142
110    0.010257
111    0.014421
112    0.007797
113    0.014688
Name: Subjectivity_score, Length: 114, dtype: float64

## Output Data

In [90]:
data = ['URL_ID','URL','Positive_score','Negative_score','Polarity_score','Subjectivity_score',
             'Average_Sentence_Length','Percentage_of_Complex_words',
             'Fog_Index','Average_number_of_words_per_sentence','complex_word_count',
             'word_count','Syllable_Count_Per_Word','personal_pronoun_count','Avg_Word_length']

In [99]:
output_data = df.drop(['text', 't_text','s_text'], axis=1)

In [100]:
output_data = output_data.reindex(columns=data)

In [101]:
 URL_ID 24,37,108 does not exists i,e. page does not exist, throughs 404 error
# so we are going to drop these rows from the table

output_data.drop([24,37,108],axis= 0, inplace=True)

In [104]:
output_data.to_excel('C:/Users/user/OneDrive/Desktop/Output_Data.xlsx')

In [105]:
output_data.head(10)

,URL_ID,URL,Positive_score,Negative_score,Polarity_score,Subjectivity_score,Average_Sentence_Length,Percentage_of_Complex_words,Fog_Index,Average_number_of_words_per_sentence,complex_word_count,word_count,Syllable_Count_Per_Word,personal_pronoun_count,Avg_Word_length
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,80,23,0.553398,0.012268,12.512500,0.587413,5.239965,12.512500,588,1001,2.678322,2,8.387612
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,38,13,0.490196,0.017647,13.280000,0.635542,5.566217,13.280000,211,332,2.629518,3,8.704819
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,18,27,-0.200000,0.009053,8.884058,0.517129,3.760475,8.884058,317,613,2.199021,3,8.109299
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,35,26,0.147541,0.009992,12.433333,0.529491,5.185130,12.433333,395,746,2.325737,7,8.183646
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,35,26,0.147541,0.009992,12.433333,0.529491,5.185130,12.433333,395,746,2.325737,7,8.183646
5,2893.8,https://insights.blackcoffer.com/rise-of-chatb...,48,11,0.627119,0.010481,10.523077,0.516082,4.415664,10.523077,353,684,2.305556,3,8.229532
6,3355.6,https://insights.blackcoffer.com/rise-of-e-hea...,32,10,0.523810,0.007902,13.955556,0.584395,5.815980,13.955556,367,628,2.445860,1,8.463376
7,3817.4,https://insights.blackcoffer.com/how-does-mark...,59,6,0.815385,0.008181,10.168421,0.526915,4.278134,10.168421,509,966,2.342650,5,8.224638
8,4279.2,https://insights.blackcoffer.com/how-advertise...,7,0,1.000000,0.003341,10.230769,0.451128,4.272759,10.230769,120,266,2.304511,1,7.875940
9,4741.0,https://insights.blackcoffer.com/negative-effe...,19,46,-0.415385,0.012700,12.280000,0.576547,5.142619,12.280000,354,614,2.434853,6,8.335505
